# Modelos ViT pré-treinados

### Biblioteca Pytorch Image Models (timm)

In [23]:
import torch
import timm

tag = 'vit_base_patch16_224'

model = timm.create_model(tag, pretrained=True)
model.eval()

x = torch.rand(1, 3, 224, 224)
with torch.no_grad():
    scores = model(x)
scores.shape

torch.Size([1, 1000])

In [24]:
model = timm.create_model(tag, pretrained=True, num_classes=0)
model.eval()

with torch.no_grad():
    scores = model(x)
scores.shape

torch.Size([1, 768])

### Modelos treinados no ImageNet

How to train your ViT? Data, Augmentation, and Regularization in Vision Transformers

https://arxiv.org/abs/2106.10270

* ImageNet 1k: 1281167 imagens separadas em 1000 classes
* ImageNet 21k: 14197122 images separadas em 21841 classes

In [25]:
tag = 'vit_base_patch16_224.augreg_in1k'
tag = 'vit_base_patch16_224.augreg_in21k_ft_in1k'

### Treinamento para recuperar regiões apagadas de imagens

Masked Autoencoders Are Scalable Vision Learners

https://arxiv.org/abs/2111.06377

Treinado no ImageNet 1k

![](../data/notebook_images/mae.png)

In [ ]:
tag = 'vit_base_patch16_224.mae'

### Similaridade entre imagens transformadas

Emerging Properties in Self-Supervised Vision Transformers

https://arxiv.org/pdf/2104.14294

A Simple Framework for Contrastive Learning of Visual Representations

https://arxiv.org/abs/2002.05709

![](../data/notebook_images/SimCLR.png)

In [1]:
tag = 'vit_base_patch14_reg4_dinov2.lvd142m'

### Similaridade entre textos e imagens

Learning Transferable Visual Models From Natural Language Supervision

https://arxiv.org/abs/2103.00020

* OpenAI: 400 milhões de pares de textos e imagens
* OpenCLIP: 2.32 **bilhões** de pares de textos e imagens

In [2]:
tag = 'vit_base_patch16_clip_224.laion2b_ft_in12k_in1k'
tag = 'vit_base_patch16_clip_224.openai_ft_in12k_in1k'